In [67]:
import pandas as pd
import numpy as np
from random import randint

In [68]:
# Get Data from File
csv_filename = "qc aggregated data.csv"
df = pd.read_csv(csv_filename)

# Specify Input and Output from Data
input_columns = ["tempmax", "tempmin", "temp", "feelslikemax", "feelslikemin", "feelslike", "dew", "humidity", "windspeed"]
output_column = ["conditions"]
X = df[input_columns]
Y = df[output_column]

In [69]:
max_count = 0
all_conditions = list(df.conditions.unique())
all_conditions

['Partially cloudy', 'Rain, Partially cloudy', 'Rain, Overcast', 'Overcast']

In [70]:
for condition in df.conditions.unique():
    num_per_class = len(df[df.conditions == condition])
    print(f"{condition} : {num_per_class}")
    if num_per_class > max_count:
        max_count = num_per_class

Partially cloudy : 695
Rain, Partially cloudy : 1216
Rain, Overcast : 124
Overcast : 3


In [77]:
t_l = df[df.conditions == all_conditions[0]][input_columns + output_column].to_numpy().tolist()
t_l

[[29.0, 21.7, 26.2, 31.7, 21.7, 27.4, 20.6, 72.0, 22.1, 'Partially cloudy'],
 [32.5, 23.8, 28.0, 36.7, 23.8, 30.6, 23.7, 78.8, 21.9, 'Partially cloudy'],
 [33.0, 25.5, 28.5, 35.1, 25.5, 31.0, 22.0, 68.8, 20.3, 'Partially cloudy'],
 [32.0, 23.8, 28.2, 35.9, 23.8, 30.6, 23.4, 77.4, 18.1, 'Partially cloudy'],
 [29.6, 24.7, 27.0, 31.4, 24.7, 28.0, 20.3, 67.5, 12.8, 'Partially cloudy'],
 [29.0, 24.2, 26.3, 29.9, 24.2, 26.7, 18.4, 62.7, 25.6, 'Partially cloudy'],
 [30.0, 23.2, 26.3, 30.8, 23.2, 26.6, 18.0, 60.9, 31.3, 'Partially cloudy'],
 [30.0, 22.0, 26.1, 31.3, 22.0, 26.6, 19.3, 67.1, 25.5, 'Partially cloudy'],
 [31.0, 22.9, 26.6, 32.4, 22.9, 27.2, 20.0, 68.8, 16.3, 'Partially cloudy'],
 [32.0, 20.4, 27.2, 34.1, 20.4, 28.0, 21.0, 70.9, 16.7, 'Partially cloudy'],
 [31.0, 22.8, 27.0, 34.8, 22.8, 28.8, 22.5, 77.9, 16.3, 'Partially cloudy'],
 [30.0, 23.3, 27.0, 34.8, 23.3, 28.3, 22.1, 75.6, 16.4, 'Partially cloudy'],
 [29.0, 22.3, 26.0, 30.8, 22.3, 26.6, 21.0, 75.4, 12.8, 'Partially cloudy'],

In [79]:
len(t_l), max_count - len(t_l)

(695, 521)

In [83]:
r_i = randint(0, len(t_l))
t_l.append(t_l[r_i])
len(t_l), max_count - len(t_l)

(699, 517)